[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/SuperbTUM/Modelshare_Development/blob/master/notebooks/Custom_RESTAPI_Lambda_Function.ipynb)

In [ ]:
!pip install aimodelshare --upgrade -q

     |████████████████████████████████| 951 kB 28.6 MB/s 
     |████████████████████████████████| 4.5 MB 47.6 MB/s 
     |████████████████████████████████| 132 kB 45.9 MB/s 
     |████████████████████████████████| 291 kB 45.7 MB/s 
     |████████████████████████████████| 288 kB 59.3 MB/s 
     |████████████████████████████████| 442 kB 50.5 MB/s 
     |████████████████████████████████| 146 kB 74.4 MB/s 
     |████████████████████████████████| 13.1 MB 44.9 MB/s 
     |████████████████████████████████| 83 kB 2.3 MB/s 
     |████████████████████████████████| 308 kB 64.3 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 178 kB 61.1 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 8.6 MB 55.2 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 140 kB 56.5 MB/s 
     |████████████████████████████████| 55 kB 144 kB/s 
     |███████████████

In [ ]:
import aimodelshare as ai

In [ ]:
from aimodelshare.aws import configure_credentials

# Why not authenticate here?
configure_credentials()

AI Modelshare Username:··········
AI Modelshare Password:··········
AWS_ACCESS_KEY_ID:··········
AWS_SECRET_ACCESS_KEY:··········
AWS_REGION:··········
Configuration successful. New credentials file saved as 'credentials.txt'


In [ ]:
!ls

credentials.txt  sample_data


In [ ]:
# Execute this before creating a competition
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model")

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [ ]:
X_train, X_test, y_train_labels, y_test_labels, example_data = ai.import_quickstart_data("covid_tweets")
# Each sample is a sentence
print(len(X_train))


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
6505


Load text dataset

In [ ]:
# Transform text to embeddings
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)

def preprocessor(data, maxlen=40, max_words=10000):

    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

X_train_processed = preprocessor(X_train)
X_test_processed = preprocessor(X_test)

In [ ]:
!ls

covid_tweet_competition_data  model.onnx		    sample_data
credentials.txt		      quickstart_covid_competition


Train a model for one epoch

In [ ]:
# Let's work with a keras model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten
import pandas as pd

model = Sequential()
model.add(Embedding(10000, 16, input_length=40))
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
y_train = pd.get_dummies(y_train_labels)

history = model.fit(X_train_processed, y_train,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.2)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 16)            160000    
                                                                 
 flatten_5 (Flatten)         (None, 640)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 1282      
                                                                 
Total params: 161,282
Trainable params: 161,282
Non-trainable params: 0
_________________________________________________________________
163/163 [==============================] - 2s 5ms/step - loss: 0.5037 - acc: 0.7552 - val_loss: 0.3762 - val_acc: 0.8294


In [ ]:
# Save sklearn model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

# Check how many preprocessed input features there are
from skl2onnx.common.data_types import FloatTensorType
feature_count = preprocessor(X_test).shape[1]
initial_type = [('float_input', FloatTensorType([None, feature_count]))]  # Insert correct number of preprocessed features

onnx_model = model_to_onnx(model, 
              framework='keras',
              initial_types=initial_type, 
              transfer_learning=False,
              deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

Create a competition

In [ ]:
from aimodelshare import ModelPlayground

# Instantiate ModelPlayground() Class
# Only if the model exists
# Deploy online model first?
myplayground = ModelPlayground(model_type="text", 
                classification=True, 
                private=False)
model_filepath = "model.onnx"
# The command checks the path validity of model, but not the preprocessing
preprocessor_filepath = "preprocessor.zip"
myplayground.deploy(model_filepath, preprocessor_filepath, y_train_labels, example_data)

# Deploy online competition
myplayground.create_competition(data_directory='covid_tweet_competition_data', 
                y_test=y_test_labels, 
                public=True)

We need some information about your model before we can build your REST API and interactive Model Playground.
   
Model Name (for AI Model Share Website):text model
Model Description (Explain what your model does and 
 why end-users would find your model useful):text classification task
Model Key Words (Search categories that describe your model, separated with commas):text classfication
   
Creating your prediction API. (This process may take several minutes.)

[===                                  ] Progress: 5% - Accessing Amazon Web Services, uploading resources...The preprocessor file at preprocessor.zip does not exist
[=====================================] Progress: 100% - Complete!                                            

Success! Your Model Playground was created in 66 seconds. 
 Playground Url: "https://heo6qayusj.execute-api.us-east-2.amazonaws.com/prod/m"

You can now use your Model Playground.

Follow this link to explore your Model Playground's functionality
You can m

Submit onnx model

In [ ]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"")
prep=ai.import_preprocessor("preprocessor.zip")
prep(example_data).shape

Your preprocessor is now saved to 'preprocessor.zip'


(5, 40)

In [ ]:
from aimodelshare import ModelPlayground

myplayground = ModelPlayground(model_type="text", 
                classification=True, 
                private=False)
myplayground.playground_url = 'https://heo6qayusj.execute-api.us-east-2.amazonaws.com/prod/m'

mycompetition = ai.Competition(myplayground.playground_url)

# apiurl = myplayground.playground_url # example url from deployed playground: apiurl= "https://123456.execute-api.us-east-1.amazonaws.com/prod/m
set_credentials(apiurl='https://heo6qayusj.execute-api.us-east-2.amazonaws.com/prod/m')

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [ ]:
prediction_labels = model.predict(preprocessor(X_test))
labels = ["fake", "true"]
prediction_labels = list(map(lambda x: labels[x.argmax()], prediction_labels))
print(prediction_labels)

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx", preprocessor_filepath="preprocessor.zip",
             prediction_submission=prediction_labels)

65/65 [==============================] - 0s 1ms/step
['fake', 'true', 'fake', 'true', 'fake', 'true', 'true', 'true', 'fake', 'true', 'fake', 'fake', 'fake', 'true', 'fake', 'true', 'true', 'fake', 'fake', 'fake', 'true', 'fake', 'fake', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'fake', 'fake', 'true', 'true', 'fake', 'fake', 'true', 'true', 'true', 'true', 'true', 'fake', 'true', 'fake', 'true', 'true', 'true', 'true', 'true', 'fake', 'fake', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'true', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'true', 'fake', 'true', 'true', 'true', 'true', 'fake', 'fake', 'fake', 'fake', 'fake', 'fake', 'true', 'true', 'fake', 'true', 'fake', 'fake', 'true', 'true', 'true', 'fake', 'true', 'true', 'true', 'fake', 'fake', 'fake', 'true', 'fake', 'fake', 'fake', 'fake', 'true', 'fake', 'fake', 'true', 'true', 'true', 'fake', 'true', 'true', 'true', 'true', 'fake', 'true', 'true', 't

Load latest model

In [ ]:
latest_model = mycompetition.replicate_model(version=2) # It keeps original model structure
latest_model.summary()

Reproducibility environment is not found
Instantiate the model from metadata..
Your model is successfully instantiated.
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 16)            160000    
                                                                 
 flatten_5 (Flatten)         (None, 640)               0         
                                                                 
 dense_7 (Dense)             (None, 2)                 1282      
                                                                 
Total params: 161,282
Trainable params: 161,282
Non-trainable params: 0
_________________________________________________________________


Transfer learning

In [ ]:
# Load another text dataset

X_train, X_test, y_train_labels, y_test_labels, example_data,_, _ = ai.import_quickstart_data("imdb")
X_train = X_train[:1000]
X_test = X_test[:1000]
y_train_labels = y_train_labels[:1000]
y_test_labels = y_test_labels[:1000]
print(y_train_labels)


Data downloaded successfully.

Preparing downloaded files for use...

Success! Your Quick Start materials have been downloaded. 
You are now ready to run the tutorial.
0      positive
1      positive
2      negative
3      negative
4      positive
         ...   
995    positive
996    negative
997    negative
998    positive
999    negative
Name: label, Length: 1000, dtype: object


In [ ]:
latest_model.predict(preprocessor(X_test))

32/32 [==============================] - 0s 2ms/step


array([[0.49979478, 0.5002052 ],
       [0.50664365, 0.4933563 ],
       [0.4962984 , 0.5037016 ],
       ...,
       [0.5164275 , 0.48357248],
       [0.48962247, 0.5103774 ],
       [0.5006804 , 0.4993196 ]], dtype=float32)

Deploy RESTFUL API

In [ ]:
deployment_folder = 'deploy'

import os
import shutil
if os.path.isdir(deployment_folder):
    shutil.rmtree(deployment_folder)
os.mkdir(deployment_folder)

input_json_exampledata = {
    "data": "string"
    }
  
output_json_exampledata = ["label"]

In [ ]:
!ls

covid_tweet_competition_data  model.onnx
credentials.txt		      preprocessor.zip
deploy			      quickstart_covid_competition
imdb_competition_data	      sample_data
imdb_quickstart_materials


In [ ]:
# This is crucial
shutil.copy2("./model.onnx", "./deploy/model.onnx")

'./deploy/model.onnx'

In [ ]:
custom_lambda_handler_code = """

# Importing all the required libraries
import numpy as np
import pandas as pd
import os
import six
import json
import onnxruntime as rt

# load the ML model in global environment for faster inference time for subsequent function calls
model_file_path = 'model.onnx'
model = rt.InferenceSession('./' + model_file_path)


# Transform text to embeddings
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)

def preprocessor(data, maxlen=40, max_words=10000):

    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X

# this is the predict function to preprocess text data and generate predictions from our model
def predict(text_data):

    # labels corresponding to the index that is predicted
    labels = ['negative', 'positive']

    # preprocessing the input data
    input_data = preprocessor(text_data)
    
    # generate prediction
    input_name = model.get_inputs()[0].name
    res = model.run(None, {input_name: input_data.astype(np.float32)})

    result = list(map(lambda x: labels[x.argmax()], res))

    return result

### event['body'] is the input JSON data from REST API request ###

def handler(event, context):

    # Contents of 'body' key is string, converting content to JSON object
    body = event['body']

    if isinstance(body, six.string_types):
        body = json.loads(body)

    # Fetching image in the form of a string stored within 'data' key of body
    data = body['data']
    sample = data

    # Predict function requires path to image file as argument, saving data to file
    # temp_file = "/tmp/temp_file"
    # with open(temp_file, "wb") as fh:
    #     fh.write(sample)

    # Calling the  predict function
    result = predict(sample)

    # Deleting the image file created by the user for prediction
    # os.remove(temp_file)

    # # Returning predictions to REST API output
    result_dict = {
        "statusCode": 200,
        "headers": {
            "Access-Control-Allow-Origin" : "*",
            "Access-Control-Allow-Credentials": True,
            "Allow" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Methods" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Headers" : "*"
        },
        "body":  json.dumps(result)
    }
    return result_dict 

"""

In [ ]:
import numpy as np
import pandas as pd
import os
import six
import base64
import json
import onnxruntime as rt

# load the ML model in global environment for faster inference time for subsequent function calls
model_file_path = 'model.onnx'
model = rt.InferenceSession('./' + model_file_path)


# Transform text to embeddings
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Build vocabulary from training text data
tokenizer = Tokenizer(num_words=10000)

def preprocessor(data, maxlen=40, max_words=10000):

    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)

    word_index = tokenizer.word_index
    X = pad_sequences(sequences, maxlen=maxlen)

    return X


# this is the predict function to preprocess text data and generate predictions from our model
def predict(text_data):

    # labels corresponding to the index that is predicted
    labels = ['negative', 'positive']

    # preprocessing the input data
    input_data = preprocessor(text_data)
    
    # generate prediction
    input_name = model.get_inputs()[0].name
    res = model.run(None, {input_name: input_data.astype(np.float32)})
    prob = res[0]
    index = prob.argmax(axis=-1)

    result = list(map(lambda x: labels[x.argmax()], index))

    return result

### event['body'] is the input JSON data from REST API request ###

def handler(event, context):

    # Contents of 'body' key is string, converting content to JSON object
    body = event['body']

    if isinstance(body, six.string_types):
        body = json.loads(body)

    # Fetching image in the form of a string stored within 'data' key of body
    sample = body['data']
    # sample = base64.b64decode(data)

    # Predict function requires path to image file as argument, saving data to file
    # temp_file = "/tmp/temp_file"
    # with open(temp_file, "wb") as fh:
    #     fh.write(sample)

    # Calling the  predict function
    result = predict(sample)

    # Deleting the image file created by the user for prediction
    # os.remove(temp_file)

    # # Returning predictions to REST API output
    result_dict = {
        "statusCode": 200,
        "headers": {
            "Access-Control-Allow-Origin" : "*",
            "Access-Control-Allow-Credentials": True,
            "Allow" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Methods" : "GET, OPTIONS, POST",
            "Access-Control-Allow-Headers" : "*"
        },
        "body":  json.dumps(result)
    }
    return result_dict 

event = {"body": json.dumps({"data": ["I love Python"]})}
handler(event, None)

{'statusCode': 200,
 'headers': {'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Credentials': True,
  'Allow': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Methods': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Headers': '*'},
 'body': '["negative"]'}

In [ ]:
with open("custom_lambda_handler.py", "w") as handler_file:
    handler_file.writelines(custom_lambda_handler_code)

In [ ]:
from aimodelshare.aws import set_credentials
set_credentials(credential_file="credentials.txt", type="deploy_model", manual=False)

AI Model Share login credentials set successfully.
AWS credentials set successfully.


In [ ]:
custom_lambda_filepath = "custom_lambda_handler.py"
deployment_dir = "deploy"
private = "FALSE"
custom_libraries = "onnxruntime,numpy,pandas,pillow,tensorflow"

from aimodelshare.deploy_custom_lambda import deploy_custom_lambda
deploy_custom_lambda(input_json_exampledata,
                     output_json_exampledata,
                     custom_lambda_filepath,
                     deployment_dir,
                     private,
                     custom_libraries=custom_libraries)

[=====================================] Progress: 100% - API deployment completed!                          

We need some information about your model before we can generate your API.

Name your deployment: ming22
Describe your deployment: ming22_demo
Describe your deployment's performance (OPTIONAL): 
Enter comma-separated search categories for your deployment (OPTIONAL): 

Your AI Model Share API was created in 478 seconds. API Url: "https://x1sfbkty10.execute-api.us-east-2.amazonaws.com/prod/m"



Follow this link to explore your Model Playground's functionality
You can make predictions with the cURL functionality and access example code from the Programmatic tab.
https://www.modelshare.org/detail/{"curlcolabhtmllinkcode": "", "delete": "FALSE", "searchitems": "superbtum_ming22_demo__custom_https://x1sfbkty10.execute-api.us-east-2.amazonaws.com/prod/m_", "input_shape": "", "apicalls": "0", "apimodeltype": "custom", "createdAt": "1666881969789", "Private": "FALSE", "version": "1", "

In [ ]:
import base64
import requests
import json

api_url = "https://x1sfbkty10.execute-api.us-east-2.amazonaws.com/prod/m"


# Import example flower image and base64 encode
# with open("/content/quickstart_materials/example_data/100080576_f52e8ee070_n.jpg", "rb") as image_file:
#   encoded_string = base64.b64encode(image_file.read())
#   data = json.dumps({"data": encoded_string.decode('utf-8') })

data = json.dumps({"data": ["I love Python"]})
print(data)

# Set up authorization headers and add token (Your API token is available in your playground page...
# under predict > programmatic)
headers = {"Content-Type": "application/json", 
      "authorizationToken": "eyJraWQiOiIxRHBcL2FMakJvNmozdHRHZFd6dEVEbUR5V0FPN3JtVEVyaHRDRnltQmlVST0iLCJhbGciOiJSUzI1NiJ9.eyJjdXN0b206b3JnYW5pemF0aW9uIjoiQ29sdW1iaWEgVW5pdmVyc2l0eSIsInN1YiI6IjUxOGE3ODY5LWZlOTUtNDljNC1hNGIzLTZmZTUwMDNjYzVkYiIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0yLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMl9YM0JURzc4ZzIiLCJwaG9uZV9udW1iZXJfdmVyaWZpZWQiOmZhbHNlLCJjb2duaXRvOnVzZXJuYW1lIjoiU3VwZXJiVFVNIiwiYXVkIjoiMjV2c3NibmVkMmJiYW9pMXE3cnM0aTkxNHUiLCJldmVudF9pZCI6IjIxZjljZGE0LWIwMjItNGY3YS04NDI4LWE5ZDBkNDUzMzhkYiIsInRva2VuX3VzZSI6ImlkIiwiYXV0aF90aW1lIjoxNjY2ODc2NzIwLCJuYW1lIjoiTWluZ3poZSBIdSIsImN1c3RvbTpnaXRodWIiOiJodHRwczpcL1wvZ2l0aHViLmNvbVwvU3VwZXJiVFVNIiwicGhvbmVfbnVtYmVyIjoiKzE2NDY5MjUwNzk0IiwiZXhwIjoxNjY2OTY1NzE4LCJpYXQiOjE2NjY4NzkzMTksImVtYWlsIjoibWg0MTE2QGNvbHVtYmlhLmVkdSJ9.ZIG56lchJRB11TWS_ZsfoucFaK9zoXC_lHq0rzRmBXjid0uAVxggV60c3iZXziX4Ht33ao5COM9gBaOM6ev5Q9-j0uBcg0e--BycmCNJqLUf717YznFnNp8dK2h5MwugB68b2mOZCbHckXsHv-sBJwz0MeylEHDb4ilbmt1OMhBLcXD6JLH5USsZWuXenEsMEYE7kpTufcd9Fenl0mpgdLgt1G0rcg9ccIuTJXPL-2AK1b_6RU8KLelWjTBcqdOE0pMqfbZwhKX9Hq8ZhDlSDKJ5c114r6AI42lX45Rvusg51GiL8SnlfPViQaf_78YAmgEVrPDXlC14iSSx7yIyXg" }

prediction = requests.request("POST", api_url, headers = headers, data=data)

# Print prediction
json.loads(prediction.text) #?

{"data": ["I love Python"]}


{'statusCode': 200,
 'headers': {'Access-Control-Allow-Origin': '*',
  'Access-Control-Allow-Credentials': True,
  'Allow': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Methods': 'GET, OPTIONS, POST',
  'Access-Control-Allow-Headers': '*'},
 'body': '["negative"]'}

Delete from backend: a specific or old one

In [ ]:
from __future__ import absolute_import, print_function, unicode_literals
import boto3
from datetime import datetime


client = boto3.client('lambda', region_name="us-east-2")
functions = sorted(client.list_functions()['Functions'], key=lambda x: x["LastModified"])
format = "%Y-%m-%dT%H:%M:%S.%f+0000"
threshold_time = datetime.strptime(functions[-(len(functions)>>1)]["LastModified"], format)
for function in functions:
    if threshold_time > datetime.strptime(function["LastModified"], format):
        arn = function['FunctionArn']
        print('delete_function(FunctionName={})'.format(arn))
        client.delete_function(FunctionName=arn)  # uncomment me once you've checked
    else:
        versions = client.list_versions_by_function(FunctionName=function['FunctionArn'])['Versions']
        for version in versions:
            if version['Version'] != function['Version']:
                arn = version['FunctionArn']
                print('delete_function(FunctionName={})'.format(arn))
                client.delete_function(FunctionName=arn)  # uncomment me once you've checked

print("-----------------after delete")
for function in functions:
  print(function)

delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:evalfunctionYDkgDXsS33T3aadWhGFBYE)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:modfunctionUnRQ3GZpQNHTcmQyC2RjrA)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:evalfunctionjsKDAZ2Y6o2d2La8Cpz6RA)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:redisAccessjsKDAZ2Y6o2d2La8Cpz6RA)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:evalfunctionUjg6FgXGvUJyjYXrDj64mW)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:redisAccessUjg6FgXGvUJyjYXrDj64mW)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:redisAccessLBJAdyHDQF39Tx9PEt9XQd)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:redisAccessThnEjCQimra2GG67yGvo2d)
delete_function(FunctionName=arn:aws:lambda:us-east-2:165928433299:function:evalfunctiongvLh2TRixogABkeaCacve